# Regression

In [ ]:
import learning
from IPython.core import display as ICD
import pandas as pd
from mord import OrdinalRidge
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.neural_network import MLPRegressor
pd.options.display.max_columns = 999
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_regression
from tqdm import tqdm

models = [
    LinearRegression(),
    MLPRegressor(), 
    Ridge(),
    Lasso(),
    MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(20, 20), random_state=1),
    KNeighborsClassifier(1),
    DecisionTreeClassifier(max_depth=10),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    RandomForestClassifier(max_depth=10, n_estimators=30, max_features=1),
    MLPClassifier(alpha=1),
    AdaBoostClassifier(),
    AdaBoostClassifier(n_estimators=100),
    AdaBoostClassifier(n_estimators=200),
    GaussianNB()]

def trykfeatures(output, train, test, model):
    _, Xtrain, ytrain = learning.prepare_base_data(train, output=output)
    _, Xtest, ytest = learning.prepare_base_data(test, output=output)
    minscore = 10000
    for k in range(1, 17):
        skb = SelectKBest(f_regression, k=k)
        skb.fit(Xtrain, ytrain)
        Xt = skb.transform(Xtrain)
        model.fit(Xt, ytrain)
        trues, preds, score = learning.score(test, {output: model}, {output: skb})
        if(score<minscore):
            finalskb = skb
            finalmodel = model
            minscore = score
    return finalskb, finalmodel, minscore

def trymodels(output, train, test, models):
    print('Finding ' + output)
    minscore = 10000
    for model in tqdm(models):
        finalskb, finalmodel, modelminscore = trykfeatures(output, train, test, model)
        if(modelminscore<minscore):
            chosenmodel = finalmodel
            chosenskb = finalskb
            minscore = modelminscore
    return chosenmodel, chosenskb, minscore

train = learning.get_base_data(2015)
test = learning.get_base_data(2014)

outputs = ["Goals", "Assists", "Shots", "Blocks"]
fmodels = {}
fskbs = {}
fscores = {}

print('Starting...')
for output in outputs:
    m, skb, score = trymodels(output, train, test, models)
    print(output)
    print(m)
    print(skb)
    print(score)
    fmodels[output] = m
    fskbs[output] = skb
    fscores[output] = score
    
true, pred, score = learning.score(test, fmodels, featuresd=fskbs)
print(score)

Starting...
Finding Goals


 21%|██▏       | 3/14 [01:51<06:18, 34.43s/it]

In [ ]:
%autoreload 2
import learning
for key, value in fskbs.items():
    print(key)
    print(value)
true, pred, score = learning.score(test, fmodels, featuresd=fskbs)
print(score)